In [6]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

In [7]:
submission_df = pd.read_csv("y_preds.csv")
df_test = pd.read_csv('test_amended.csv')

In [8]:
df_test_non = pd.read_csv('test.csv')

In [9]:
# one of the interpolations was placed below 0.7 which makes the upload break
df_test.iloc[889,1] = 0.75

In [10]:
df_test

,galactic year,existence expectancy index,existence expectancy at birth,Gross income per capita,Income Index,Expected years of education (galactic years),Mean years of education (galactic years),Intergalactic Development Index (IDI),Education Index,"Intergalactic Development Index (IDI), Rank",...,galaxy_UGCA 292,galaxy_UGCA 438 (ESO 407-018),galaxy_UGCA 86,galaxy_UGCA 92,galaxy_Ursa Major I Dwarf (UMa I dSph),galaxy_Ursa Major II Dwarf,galaxy_Ursa Minor Dwarf,galaxy_Virgo I,galaxy_Willman 1,"galaxy_Wolf-Lundmark-Melotte (WLM, DDO 221)"
0,1007012,0.456086,51.562543,12236.576447,0.593325,10.414164,10.699072,0.547114,0.556267,232.621842,...,0,0,0,0,0,0,0,0,0,0
1,1007012,0.529835,57.228262,3431.883825,0.675407,7.239485,5.311122,0.497688,0.409969,247.580771,...,0,0,0,0,0,0,0,0,0,0
2,1008016,0.560976,59.379539,27562.914252,0.594624,11.774890,5.937797,0.544744,0.486167,249.798771,...,0,0,0,0,0,0,0,0,0,0
3,1007012,0.565910,59.952390,20352.232905,0.837700,11.613621,10.067882,0.691641,0.523441,211.505060,...,0,0,0,0,0,0,0,0,0,0
4,1013042,0.588274,55.428320,23959.704016,0.520579,10.392416,6.374637,0.530676,0.580418,234.721069,...,0,0,0,0,0,0,0,1,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
885,1016064,0.859797,75.291914,18170.157548,0.707996,11.154777,4.190958,0.701760,0.540954,212.874869,...,0,0,0,0,0,0,0,0,0,0
886,1016064,0.912304,72.601017,19767.681597,0.715522,13.126625,10.552630,0.732641,0.622619,190.209811,...,0,0,0,0,0,0,0,0,0,0
887,1016064,1.090993,88.276857,31700.561656,0.957858,19.425120,11.120741,1.092724,0.974172,128.115014,...,0,0,0,0,0,0,0,0,0,0
888,1016064,1.005144,82.100572,25561.963151,0.751660,16.198364,13.756741,0.819249,0.884783,156.160418,...,0,0,0,0,0,0,0,0,0,0


In [11]:
# creating the energy allocation - mean allocation

# amount for galaxies with amount below 0.7
bottom10 = 50000*0.1/len(df_test[df_test['existence expectancy index']<0.7])
topenergy = 50000*0.9/len(df_test[~(df_test['existence expectancy index']<0.7)])
print(bottom10,topenergy)

75.75757575757575 54.61165048543689


In [12]:
energy_allocation = df_test['existence expectancy index'].apply(lambda x: bottom10 if x<0.7 else topenergy)

In [13]:
submission_df['opt_pred'] = energy_allocation

In [14]:
# submission_df.reset_index(inplace=True)
# submission_df.to_csv('submission1.csv',index=False)

In [15]:
submission_df.sort_values("pred")

,pred,opt_pred
226,0.014767,54.611650
49,0.016446,75.757576
37,0.019695,75.757576
34,0.019734,75.757576
26,0.019741,75.757576
...,...,...
705,0.455710,54.611650
800,0.456015,54.611650
801,0.456195,54.611650
583,0.680306,54.611650


## Playing around with optimisation

In [16]:
submission_df['index_increase'] = submission_df.opt_pred*(-np.log(submission_df.pred+0.01)+3)**2/1000

In [17]:
# checking the index increase for submission one 
submission_df.index_increase.sum()

1598.697747905815

In [18]:
test1 = submission_df.sort_values("pred")

In [19]:
df_test[df_test['existence expectancy index']<0.7].index

Int64Index([ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16,
            17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33,
            34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50,
            51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65],
           dtype='int64')

In [20]:
# setting values for submission 2
test1.opt_pred.iloc[:500] = 100
test1.opt_pred.iloc[500:] = 0
test1['index_increase'] = test1.opt_pred*(-np.log(submission_df.pred+0.01)+3)**2/1000
test1.index_increase.sum()

1813.7720476017962

In [21]:
test1[test1.index.isin(df_test[df_test['existence expectancy index']<0.7].index)].sum()

pred                 2.391142
opt_pred          6400.000000
index_increase     238.808020
dtype: float64

In [22]:
test1[499:501]

,pred,opt_pred,index_increase
748,0.055734,100.0,3.274287
536,0.056166,0.0,0.000000


In [23]:
test1.drop("index_increase",axis=1,inplace=True)
test1.sort_index(inplace=True)
test1

,pred,opt_pred
0,0.042977,100.0
1,0.040024,100.0
2,0.040381,100.0
3,0.039610,100.0
4,0.034555,100.0
...,...,...
885,0.028270,100.0
886,0.034195,100.0
887,0.068374,0.0
888,0.063687,0.0


In [24]:
# test1.to_csv('submission2.csv',index=False)

In [25]:
# check if there are decreasing returns - it would seem that there aren't
(100*(-np.log(0.055550+0.01)+3)**2/1000)-(99*(-np.log(0.055550+0.01)+3)**2/1000)-(1*(-np.log(0.056333+0.01)+3)**2/1000)

0.00013581836830876132

In [30]:
# setting values for submission 3
test2 = submission_df.sort_values("pred").copy()
test2.opt_pred.iloc[:400] = 100
test2.opt_pred.iloc[400:600] = 50
test2.opt_pred.iloc[600:] = 0


test2['index_increase'] = test2.opt_pred*(-np.log(submission_df.pred+0.01)+3)**2/1000
test2.index_increase.sum()

1805.111864173468

In [31]:
test2.drop("index_increase",axis=1,inplace=True)
test2.sort_index(inplace=True)
test2

,pred,opt_pred
0,0.042977,100.0
1,0.040024,100.0
2,0.040381,100.0
3,0.039610,100.0
4,0.034555,100.0
...,...,...
885,0.028270,100.0
886,0.034195,100.0
887,0.068374,0.0
888,0.063687,50.0


In [93]:
test2[test2.index.isin(df_test[df_test['existence expectancy index']<0.7].index)].sum()

index       2145.000000
pred           2.405475
opt_pred    6500.000000
dtype: float64

In [32]:
test2.reset_index(inplace=True)
test2

,index,pred,opt_pred
0,0,0.042977,100.0
1,1,0.040024,100.0
2,2,0.040381,100.0
3,3,0.039610,100.0
4,4,0.034555,100.0
...,...,...,...
885,885,0.028270,100.0
886,886,0.034195,100.0
887,887,0.068374,0.0
888,888,0.063687,50.0


In [33]:
test2.to_csv('submission4.csv',index=False)

In [47]:
# setting values for submission 5
test3 = submission_df.sort_values("pred").copy()
test3.opt_pred.iloc[:300] = 100
reduce = list(np.linspace(0,100,300))
reduce.reverse()
test3.opt_pred.iloc[350:650] = reduce
test3.opt_pred.iloc[650:] = 0


test3['index_increase'] = test2.opt_pred*(-np.log(submission_df.pred+0.01)+3)**2/1000
test3.index_increase.sum()

1805.111864173468

In [48]:
test3.drop("index_increase",axis=1,inplace=True)
test3.sort_index(inplace=True)
test3.reset_index(inplace=True)
test3

,index,pred,opt_pred
0,0,0.042977,75.757576
1,1,0.040024,100.000000
2,2,0.040381,100.000000
3,3,0.039610,100.000000
4,4,0.034555,100.000000
...,...,...,...
885,885,0.028270,100.000000
886,886,0.034195,100.000000
887,887,0.068374,14.381271
888,888,0.063687,27.090301


In [49]:
test3.to_csv('submission5.csv',index=False)